In [2]:
import numpy as np
import scipy as sp

In [3]:
def expression_matrix(n, p):
    
    X = np.random.randn(n, p)
    #print(X)



    col_means = np.mean(X, axis=0)
    
    col_std = np.std(X, axis=0)
    
    X_normalized = (X - col_means) / col_std #We use the  normalization to have a zero mean and a unit variance 
    
    return X_normalized

In [4]:
p = 5
n = 50   

X = expression_matrix(n, p)
y = np.random.randint(2, size=p)
print(y)

print(f'Normalized values: {X}')
print(f"standard deviation: {np.std(X, axis=0)}")

[0 0 1 1 1]
Normalized values: [[ 4.49119696e-01 -2.09623176e-01  1.43693278e-01  1.13788477e+00
   5.25141447e-01]
 [ 7.11299445e-01  6.52586651e-01  2.58289655e-01  4.27311405e-01
   8.96039095e-02]
 [-5.34399947e-03  3.62987796e-01  1.02915172e+00 -6.37290570e-01
  -5.72571290e-01]
 [-1.02817507e-01 -8.89865678e-02  7.00172825e-01  5.41529135e-01
   6.75065458e-01]
 [-2.45289043e+00 -1.54961552e-01  1.72871889e+00 -5.61043443e-01
  -7.31750798e-01]
 [-1.05050861e+00 -8.00347616e-01 -3.56648086e-01 -1.10194640e+00
  -9.32964520e-01]
 [ 1.39007865e+00 -1.23944721e+00  9.71957629e-01  1.01657811e+00
  -4.69133660e-02]
 [-3.62460835e-01 -5.91294448e-01 -9.02007640e-01 -1.30687400e-02
   6.25907687e-01]
 [-1.27363403e+00 -2.68146728e-01  4.67324739e-01  9.34068145e-01
   1.32575259e-01]
 [-1.97546638e+00 -1.14465325e+00 -1.92367109e+00  7.63434485e-01
   6.35366244e-01]
 [ 1.15491633e+00  3.68702698e-04  1.06283349e+00  4.62687007e-01
   1.75300051e+00]
 [ 4.76198177e-02  3.72725422e-01 

In [5]:
def score(X, y):

    y0 = []
    y1 = []
    for index, j in enumerate(y):
        if j == 1:
            y1 = np.append(y1 ,index)
    
    y1 = np.asarray(y1, dtype = 'int')
    score_of_x = np.zeros(n)
        
    
    for i in range(p):
        if i not in y1:
            y0 = np.append(y0, i)
    y0 = np.asarray(y0, dtype = 'int')

                    
    for index, row in enumerate(X):
        for i in y1:
            for j in y0:
                if row[i] >= row[j]:
                    score_of_x[index] = score_of_x[index] + 1
       
    return score_of_x,y0,y1

In [6]:
scoreX,y0,y1 = score(X, y)

In [7]:
def flip_sign(X, score):
    score_max = np.max(score)
    
    for i in range(len(score)):
        if(score[i] > score_max/2):
            X[i] = X[i] * (-1)

    return X
        


In [8]:
X = flip_sign(X, scoreX)

In [9]:
def new_score(score):
    new_score = []
    s_max = np.max(score)
    for i in range(len(score)):
        n = np.minimum(int(score[i]), int(s_max)-int(score[i]))
        new_score.append(n)
    return new_score
        

In [10]:
newscore = new_score(scoreX)

In [11]:
def margin(X,y0,y1):
    m = []
    max_x0 = []
    min_x1 = []
    
    for i in range(X.shape[0]):
        X0 = []
        X1 = []
        for j in y0:
            X0.append(X[i][j])
        for z in y1:
            X1.append(X[i][z])
            
            
        max_x0.append(np.max(X0))
        min_x1.append(np.min(X1))
        
        m.append(min_x1[i] -max_x0[i])
        
    return m


In [12]:
def init_cluster(new_score, X, y0, y1):
    min_score = np.min(new_score)
    similar_genes = []
    margins = []
    for i in range(len(new_score)):
        if (new_score[i] == min_score):
            similar_genes.append(i)
            
    marginz = margin(X, y0, y1)
    for i in similar_genes:
        margins.append(marginz[i])
        
    max_margin = np.argmax(margins)
        
    ie = similar_genes[max_margin]
    
    return {
        "ie" : ie,
        "cg" : X[ie] 
    } 
        

In [13]:
c = []
c.append(init_cluster(newscore, X, y0, y1))

In [14]:
def average_gene(c):
    
    
    eC = np.sum(c['cg'])
    print(eC)
    eC = eC/abs(c['cg'])
 
    return eC
        

In [15]:
e_c = average_gene(c[0])

2.1390910660020026


In [16]:
def average_current_cluster(eC, c):
    
    e_Ci = []
    
    for row in X:
        sum_i = row + eC
        e_Ci.append(sum_i/abs(c['cg']+1))
 
    return e_Ci

In [17]:
e_Ci = average_current_cluster(e_c, c[0])


In [19]:
def winning_gene():
    score_e_Ci, y0, y1 = score(e_Ci, y)
    min_score = np.argmin(score_e_Ci)
    
    similar_scores = []
    
    for i in range(len(score_e_Ci)):
        if (score_e_Ci[i] == min_score):
            similar_scores.append(i)
            
    if (len(similar_scores) > 1):
        pass
        